<h3> Homework - California Housing Prices </h3>

In [31]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt

In [32]:
# Read the data
housing = pd.read_csv('housing.csv', verbose=True)

Tokenization took: 18.33 ms
Type conversion took: 14.24 ms
Parser memory cleanup took: 0.00 ms


In [33]:
housing.describe().T

,count,mean,std,min,25%,50%,75%,max
longitude,20640.0,-119.569704,2.003532,-124.3500,-121.8000,-118.4900,-118.01000,-114.3100
latitude,20640.0,35.631861,2.135952,32.5400,33.9300,34.2600,37.71000,41.9500
housing_median_age,20640.0,28.639486,12.585558,1.0000,18.0000,29.0000,37.00000,52.0000
total_rooms,20640.0,2635.763081,2181.615252,2.0000,1447.7500,2127.0000,3148.00000,39320.0000
total_bedrooms,20433.0,537.870553,421.385070,1.0000,296.0000,435.0000,647.00000,6445.0000
population,20640.0,1425.476744,1132.462122,3.0000,787.0000,1166.0000,1725.00000,35682.0000
households,20640.0,499.539680,382.329753,1.0000,280.0000,409.0000,605.00000,6082.0000
median_income,20640.0,3.870671,1.899822,0.4999,2.5634,3.5348,4.74325,15.0001
median_house_value,20640.0,206855.816909,115395.615874,14999.0000,119600.0000,179700.0000,264725.00000,500001.0000


In [34]:
housing.nunique()

longitude               844
latitude                862
housing_median_age       52
total_rooms            5926
total_bedrooms         1923
population             3888
households             1815
median_income         12928
median_house_value     3842
ocean_proximity           5
dtype: int64

In [35]:
housing.columns

Index(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income',
       'median_house_value', 'ocean_proximity'],
      dtype='object')

In [36]:
housing.isna().sum()
housing['total_bedrooms'].fillna(0, inplace=True)

In [37]:
housing.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [43]:
# Creating a new column - rooms_per_household
# housing['rooms_per_household'] = housing['households'] / housing['total_rooms'] * 100
# housing.drop('rooms_per_household', axis=1, inplace=True)
housing.insert(7, 'rooms_per_household', 
               round(housing['households'] / housing['total_rooms'] * 100, 2))

In [58]:
housing

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,bedrooms_per_room,population,households,rooms_per_household,population_per_household,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,14.66,322.0,126.0,14.32,255.56,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,15.58,2401.0,1138.0,16.03,210.98,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,12.95,496.0,177.0,12.07,280.23,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,18.45,558.0,219.0,17.19,254.79,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,17.21,565.0,259.0,15.92,218.15,3.8462,342200.0,NEAR BAY
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20635,-121.09,39.48,25.0,1665.0,374.0,22.46,845.0,330.0,19.82,256.06,1.5603,78100.0,INLAND
20636,-121.21,39.49,18.0,697.0,150.0,21.52,356.0,114.0,16.36,312.28,2.5568,77100.0,INLAND
20637,-121.22,39.43,17.0,2254.0,485.0,21.52,1007.0,433.0,19.21,232.56,1.7000,92300.0,INLAND
20638,-121.32,39.43,18.0,1860.0,409.0,21.99,741.0,349.0,18.76,212.32,1.8672,84700.0,INLAND


In [56]:
housing.drop('population_per_household', axis=1, inplace=True)


In [52]:
# Creating another column - bedrooms_per_room
housing.insert(5, 'bedrooms_per_room', \
                round(housing['total_bedrooms'] / housing['total_rooms'] * 100, 2))



In [57]:
# Creating another column - population_per_household
housing.insert(9, 'population_per_household',
               round(housing['population'] / housing['households'] * 100, 2))
